In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt

## Read in Data

In [2]:
df = pd.read_csv('Aggregated Stats.csv')
columns = df.columns
columns = columns[2:]
print(columns)
print(df.shape[0])
display(df)

Index(['TotalPopulation', 'Ozone', 'PM25', 'DieselPM', 'DrinkingWater', 'Lead',
       'Pesticides', 'ToxRelease', 'Traffic', 'CleanupSites',
       'GroundwaterThreats', 'HazWaste', 'ImpWaterBodies', 'SolidWaste',
       'Asthma', 'LowBirthWeight', 'CardiovascularDisease', 'Education',
       'LinguisticIsolation', 'Poverty', 'Unemployment', 'HousingBurden',
       'Children10years', 'Pop1064years', 'Elderly64years', 'Hispanic',
       'White', 'AfricanAmerican', 'NativeAmerican', 'AsianAmerican',
       'OtherMultiple', 'well_count', 'median_rent'],
      dtype='object')
2343


,geoid,geoid2,TotalPopulation,Ozone,PM25,DieselPM,DrinkingWater,Lead,Pesticides,ToxRelease,...,Pop1064years,Elderly64years,Hispanic,White,AfricanAmerican,NativeAmerican,AsianAmerican,OtherMultiple,well_count,median_rent
0,1400000US06037101110,6037101110,4283,0.064647,11.302131,0.047102,726.638249,60.566361,0.000000,754.980998,...,70.37,16.97,27.74,61.08,0.44,0.07,7.80,2.87,0,1391.0
1,1400000US06037101122,6037101122,3405,0.065915,11.091433,0.027323,696.161799,38.046670,0.000000,738.952332,...,74.65,19.97,4.32,84.64,2.17,0.44,6.46,1.97,0,1924.0
2,1400000US06037101210,6037101210,6347,0.064647,11.327559,0.074966,726.638249,52.489575,0.000000,769.964335,...,76.00,9.83,41.22,47.05,3.12,0.00,7.66,0.96,0,1141.0
3,1400000US06037101220,6037101220,3702,0.064647,11.258236,0.057660,726.638249,68.579783,0.000000,784.317668,...,75.80,16.10,36.84,48.95,2.35,0.00,9.21,2.65,0,1123.0
4,1400000US06037101300,6037101300,3884,0.063506,11.011945,0.071778,726.638249,50.746457,0.000000,830.445658,...,68.00,24.20,7.65,80.48,5.28,0.00,4.99,1.60,0,2131.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2338,1400000US06037980026,6037980026,26,0.065915,10.862805,0.010980,726.638249,NaN,0.000000,733.644999,...,100.00,0.00,26.92,26.92,42.31,0.00,0.00,3.85,1,NaN
2339,1400000US06037980028,6037980028,0,0.043205,11.968660,0.710312,NaN,NaN,0.978983,3418.770142,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN
2340,1400000US06037980030,6037980030,0,0.043205,11.885822,0.353936,NaN,NaN,0.000308,15274.766600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN
2341,1400000US06037980031,6037980031,1175,0.038092,11.754468,1.902832,391.079179,NaN,0.000000,5778.626709,...,88.60,9.70,32.94,33.45,19.74,2.21,5.10,6.55,82,NaN


## Data pre-processing
Impute missing values, split into train/test data, and scale data

In [3]:
df = df.set_index(['geoid', 'geoid2'])

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
out = pd.DataFrame(imp.fit_transform(df))
out.columns = df.columns
out.index = df.index

# Train Test Split
X = out.iloc[:, :-1]
y = out.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=420)

# Scale the X values, leave the y values alone
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scale = scaler.transform(X_train)
X_test_scale = scaler.transform(X_test)

In [4]:
# Format X and y into a single dataframe and save
X_train_df = pd.DataFrame(X_train_scale, index=X_train.index, columns=X_train.columns)
X_test_df = pd.DataFrame(X_test_scale, index=X_test.index, columns=X_test.columns)

X = pd.concat((X_train_df, X_test_df))

y = pd.concat((y_train, y_test))

out_df = pd.concat([X, y], axis = 1)
out_df.columns = df.columns
out_df

out_df.to_csv('cleaned_data_3_9_22.csv')

# Reset X_train and X_test to the scaled versions
X_train = X_train_scale
X_test = X_test_scale

## PCA

In [5]:
# 2 component PCA
# X = np.concatenate((X_train_scale, X_tes))

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

print(pca.explained_variance_ratio_)

principalDf = pd.DataFrame(data = principalComponents, columns = ['pc1', 'pc2'])
principalDf.index = X.index
finalDf = pd.concat([principalDf, y], axis = 1)
finalDf

[0.2078451  0.11636547]


,,pc1,pc2,median_rent
geoid,geoid2,,,
1400000US06037212501,6037212501,0.212664,-0.277051,1057.0
1400000US06037134002,6037134002,-0.062221,0.022211,1109.0
1400000US06037570001,6037570001,-1.929312,0.011936,1298.0
1400000US06037407801,6037407801,0.392322,0.008617,1232.0
1400000US06037462800,6037462800,-2.092358,0.116866,1317.0
...,...,...,...,...
1400000US06037271200,6037271200,-1.092558,-0.386029,1604.0
1400000US06037262601,6037262601,-5.907716,0.217134,3500.0
1400000US06037221601,6037221601,2.497300,-0.347122,928.0


In [6]:
# Try to do PCR (IGNORE but can use as baseline)
# This is a pipeline of PCA -> Linear Regression
pcr = make_pipeline(PCA(0.95), LinearRegression())
pcr.fit(X_train, y_train)
pca = pcr.named_steps["pca"]

# Evaluate
print(f"PCR r-squared {pcr.score(X_test, y_test):.3f}")

PCR r-squared 0.535


In [7]:
# Try to do PCA
pca = PCA(.95)
pca.fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Output number of components and the explained variance of each component
print(f"Number of PCA components: {pca.n_components_}")
print(f"PCA explained variance in each of the {pca.n_components_} components:\n{pca.explained_variance_ratio_}")

Number of PCA components: 24
PCA explained variance in each of the 24 components:
[0.22185938 0.09250137 0.07817802 0.0576076  0.05291799 0.04957648
 0.03983862 0.03593577 0.03463802 0.03217541 0.02852467 0.02694502
 0.02378932 0.02360146 0.02173658 0.02011715 0.0179344  0.0178378
 0.0167583  0.01609987 0.01416609 0.01366595 0.01330518 0.01245916]


In [8]:
print("New training data after doing PCA:")
display(pd.DataFrame(X_train_pca))

New training data after doing PCA:


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.225526,-0.999809,-2.317564,-0.444786,-1.144839,0.210936,0.727922,1.141015,1.136908,-1.116400,...,-0.381768,-0.200163,0.097942,-0.395984,1.188811,0.135058,0.263659,-0.681736,0.089035,0.092403
1,-0.032469,-1.189432,-0.773383,0.879504,0.170275,-0.169635,-0.811257,0.106051,-0.099687,-0.387603,...,0.172788,-0.606821,0.423251,-0.117368,0.621090,-0.063787,0.229816,-0.301533,-0.162461,0.150010
2,-1.905003,-0.265250,1.195212,-0.428242,-1.026079,-0.986061,0.729311,0.652070,0.116178,-0.639842,...,-0.532116,0.300917,0.986115,-0.294879,-0.220679,-0.272080,-0.068388,-0.393032,0.075911,-0.094926
3,0.410607,-1.021380,-0.172079,0.433287,-0.125097,-0.277536,0.705266,-0.273177,-1.053015,0.106356,...,-0.512115,0.812922,0.822639,0.148756,0.222526,-0.526692,-0.311607,0.547041,0.347734,0.346372
4,-2.051850,-1.035987,-0.031545,-0.372263,-0.019209,0.562007,-0.968441,-0.390291,-0.054612,0.304149,...,-0.442802,1.148580,0.436019,0.354764,0.442324,-0.034092,-0.107716,-0.024827,-0.277474,-0.064412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564,0.386378,-0.341756,4.052642,-1.002756,-1.560751,3.345321,0.253477,-1.550091,-0.608968,-1.217861,...,-0.451144,1.379553,0.204032,0.046332,-0.923405,-0.170768,-0.160426,-1.082016,-0.593261,-0.531744
1565,4.837353,-0.412463,1.000653,-0.567674,-0.509283,0.866014,-1.545796,-0.740116,-1.048896,0.873419,...,-0.061767,-1.512909,-0.151457,-0.536488,-0.551323,0.526643,-0.668305,0.646357,-0.619427,0.514201
1566,3.210036,0.250322,-3.561847,0.525374,-1.465257,0.442807,-1.162988,-0.016994,0.981299,1.090241,...,-0.215679,-0.802963,-1.334112,0.352309,-0.120194,-0.213471,1.052133,-0.291393,0.178256,-0.572856
1567,-1.356709,2.034096,-1.420735,0.592834,0.098479,-0.886030,-0.675881,-0.531968,0.512998,-0.639662,...,0.909869,-0.101632,-0.690323,-0.411891,-0.609442,0.729444,0.030977,-0.801296,0.955727,-0.528352


In [9]:
if X_train_pca.shape[0] + X_test_pca.shape[0] == df.shape[0]:
    print("No data points lost")
else:
    print("Data points lost, check code")

No data points lost


## Regression

In [10]:
# Linear Regression
# Use X_train_pca and X_test_pca


## Resources:

1. [Scikit Learn Missing Value Imputation](http://scikit-learn.org/stable/modules/impute.html)
2. [Scikit Learn Train Test Split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
3. [Scikit Learn Standard Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)
4. [Why use standardization](https://towardsai.net/p/data-science/how-when-and-why-should-you-normalize-standardize-rescale-your-data-3f083def38ff#:~:text=When%20Should%20You%20Use%20Normalization%20And%20Standardization%3A&text=Normalization%20is%20useful%20when%20your,and%20artificial%20neural%20networks.)
5. [Scikit Learn PCA](https://scikit-learn.org/stable/auto_examples/cross_decomposition/plot_pcr_vs_pls.html)
7. [PCA Using Scikit Learn (Medium Article)](https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60)

## To-Do:

- [X] Data pre-processing
    - [X] Impute missing values
    - [X] Scale feature variables
    - [X] Train-Test Split
- [X] Feature Selection
    - [X] PCA
    - [X] Spearman Correlation Coefficient (Stefan) *did not end up going forward with using this*
- [ ] Modelling
    - [ ] Regression
        - [ ] Linear Regression
        - [ ] Other methods
    - [X] Random Forest (Stefan)
    - [ ] Neural Networks

In [11]:
X_train, X_test = X_train_pca, X_test_pca

In [12]:
X_train.shape

(1569, 24)

In [13]:
from sklearn.ensemble import RandomForestRegressor

RF_regressor = RandomForestRegressor(n_estimators = 1000, random_state=0)
RF_regressor.fit(X_train,y_train)

RandomForestRegressor(n_estimators=1000, random_state=0)

In [14]:
y_pred = RF_regressor.predict(X_test)

In [15]:
RF_regressor.score(X_test,y_test)

0.6180752628483029

In [16]:
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)),'.3f'))
print("\nRMSE:\n", rmse)


RMSE:
 305.326
